Refer to `README.md` for installation instructions.

## 1. Configuration

Let's start off by sourcing secrets and defining the locations of the various required services:

In [1]:
# source secrets
source .env

#### **2. List TES instances**

Let's see what TES instances we have defined:

In [ ]:
unset TES_INSTANCES
declare -A TES_INSTANCES
while IFS=',' read -r KEY URL; do
    TES_INSTANCES["$KEY"]=$URL
done < .tes_instances

for KEY in "${!TES_INSTANCES[@]}"; do
    echo "$KEY: ${TES_INSTANCES[$KEY]}"
done

## 3. Executing workflows via the TES network

Similar to the demo done by Alex Kanitz for cwl-tes, we will demonstrate how a workflow engine with a TES backend
can execute workflows on a different TES instances.

We will use the [`snakemake`](https://github.com/snakemake/snakemake) workflow engine.

### Running Snakemake workflows

As for cwl-tes, we will reuse the "hash splitter" workflow, a simple
workflow with a scatter-gather step. The tasks of the workflow will be executed on individula TESK/Funnnel instances.
Existing files will be overwritten.

In [ ]:
unset TASKS
declare -A TASKS
export HOME=/tmp
for KEY in "${!TES_INSTANCES[@]}"; do
    TES="${TES_INSTANCES[$KEY]}"
    echo "Submitting task to $KEY ($TES)..."
    TASK_ID=$( \
        snakemake --tes "${TES%/}" --snakefile Snakefile_s3 --jobs 1 --cores 1 -F --rerun-incomplete --envvars HOME  | \
        jq ".id" - | \
        tr -d '"'
    )
    if [[ $TASK_ID == "null" ]]; then
        echo "FAILED"
    else
        echo "Task ID: $TASK_ID"
        TASKS[[ $TASK_ID ]]="$TES"
    fi
    echo "================================================================================"
done
echo "DONE"

In [ ]:
Now lets submitt the workflow tasks to proTES.

In [ ]:
export HOME=/tmp
TES="$TES_GATEWAY" #proTES
TASK_ID=$( \
    snakemake --tes "${TES%/}/ga4gh/tes/" --snakefile Snakefile_s3 --jobs 1 --cores 1 --envvars HOME | \
    jq ".id" - | \
    tr -d '"'
)


if [[ $TASK_ID == "null" ]]; then
    echo "FAILED"
else
    echo "Task ID: $TASK_ID"
fi
echo "================================================================================"
echo "DONE"


Let's check one more time the status of a selected task:

RESPONSE=$( \
    curl \
    --silent \
    --request "GET" \
    --header "accept: application/json" \
    --header "Content-Type: application/json" \
    "${TES%/}/v1/tasks/${TASK_ID}"
)
echo $



